In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os
import random

2022-08-18 19:59:21.607942: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-18 19:59:21.607980: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
DIRECTORY = r"/home/user/ML/Deep Learning/CNN/Covid/train"
TEST = r"/home/user/ML/Deep Learning/CNN/Covid/test"

CATEGORIES = ["Covid","Normal","Viral Pneumonia"]

data = []
for category in CATEGORIES:
    folder = os.path.join(DIRECTORY,category)
    label = CATEGORIES.index(category)
#     print(folder)
    for img in os.listdir(folder):
        img = os.path.join(folder,img)
        img_arr = cv2.imread(img)
        img_arr = cv2.resize(img_arr,(100,100))
#         plt.imshow(img_arr)
#         print(img_arr)
        data.append([img_arr,label])
        

In [3]:
def get_label(index):
    if index == 0:
        return "Covid"
    if index == 1:
        return "Normal"
    if index == 2:
        return "Viral Pneumonia"
    
def show_pic(index): 
    plt.imshow(data[index][0])
    plt.xlabel(get_label(data[index][1]))
    plt.show()
    
# show_pic(5)
# show_pic(1)


In [4]:
random.shuffle(data)
# show_pic(2)

In [5]:
X = []
Y = []

for features, label in data:
    X.append(features)   
    Y.append(label)
    
    
X = np.asarray(X)
Y = np.asarray(Y)

X = X / 255
X.shape[1:]

(100, 100, 3)

In [6]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

#pre=processing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
    )

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255,
     validation_split=0.1
)

train_datagen = train_datagen.flow_from_directory(
    DIRECTORY,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'training'
)

test_datagen = test_datagen.flow_from_directory(
    TEST,
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'validation'
)

Found 226 images belonging to 3 classes.
Found 6 images belonging to 3 classes.


In [7]:
train_datagen.class_indices


{'Covid': 0, 'Normal': 1, 'Viral Pneumonia': 2}

In [8]:

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense

model = Sequential()
model.add(Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu',input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2,strides=2))

model.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

# cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

model.add(Flatten())
model.add(Dense(3,activation='softmax'))

2022-08-18 19:59:32.486451: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-18 19:59:32.486837: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-08-18 19:59:32.486963: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-08-18 19:59:32.487028: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: lib

In [9]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit(train_datagen,epochs=10,validation_data=test_datagen)

Epoch 1/10
4/4 [==============================] - 13s 3s/step - loss: 1.0727 - accuracy: 0.3982 - val_loss: 1.0320 - val_accuracy: 0.5000
Epoch 2/10
4/4 [==============================] - 12s 3s/step - loss: 0.9840 - accuracy: 0.5575 - val_loss: 0.8872 - val_accuracy: 0.6667
Epoch 3/10
4/4 [==============================] - 11s 3s/step - loss: 0.8462 - accuracy: 0.6814 - val_loss: 0.6931 - val_accuracy: 0.8333
Epoch 4/10
4/4 [==============================] - 12s 3s/step - loss: 0.6641 - accuracy: 0.7832 - val_loss: 0.6359 - val_accuracy: 0.6667
Epoch 5/10
4/4 [==============================] - 12s 3s/step - loss: 0.5880 - accuracy: 0.7920 - val_loss: 0.4895 - val_accuracy: 0.8333
Epoch 6/10
4/4 [==============================] - 12s 3s/step - loss: 0.5521 - accuracy: 0.7566 - val_loss: 0.3981 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 12s 3s/step - loss: 0.4565 - accuracy: 0.8097 - val_loss: 0.4486 - val_accuracy: 0.6667
Epoch 8/10
4/4 [==================

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 3)                 1

In [11]:
from tensorflow.keras.models import Model
a = np.array(history.history['accuracy']).mean()
# history.history
model.evaluate_generator(generator=test_datagen)

/tmp/ipykernel_19042/731998689.py:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(generator=test_datagen)


[0.3872052729129791, 0.8333333134651184]

In [12]:
from keras.preprocessing import image
path = "/home/user/ML/Deep Learning/CNN/Covid/net"
# path = "/home/user/ML/Deep Learning/CNN/Covid/test/Normal"
img = path + "/covid.png"
try:
    
    img_pred = tf.keras.utils.load_img(img,target_size=(IMAGE_SIZE,IMAGE_SIZE))
    img_pred = tf.keras.utils.img_to_array(img_pred)
    img_pred = img_pred / 255
    img_pred = np.expand_dims(img_pred, axis=0)
except:
    print("file not found!")

result = model.predict(img_pred)
# plt.imshow(img_pred)
print(result)
# print(get_label(np.argmax(result)))
print(CATEGORIES[np.argmax(result)])

1/1 [==============================] - 0s 128ms/step
[[9.9506915e-01 4.4714608e-03 4.5940385e-04]]
Covid


In [13]:
# model.evaluate(X,Y)